In [5]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

train_images,train_labels, test_images, test_labels = None, None, None, None

In [6]:
def loadDatasets():
    global train_images,train_labels, test_images, test_labels
    dbPath = os.getcwd() + r"\..\mnist\datasets\mnist.npz"
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data(path= dbPath)

    train_labels = train_labels[:1000]
    test_labels = test_labels[:1000]

    train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
    test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


In [7]:
def create_model():
      model = tf.keras.models.Sequential([
            keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(10, activation=tf.nn.softmax)])

      model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

      return model

In [8]:
def trainModel(model):
    global checkpoint_path, checkpoint_dir
    checkpoint_path = 'trainCheckPoint/cp.ckpt'
    checkpoint_dir = os.path.dirname(checkpoint_path)
    print(f"checkpoint:{checkpoint_path},checkpoint_dir:{checkpoint_dir}")
    
    # Create checkpoint callback
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    model = create_model()

    model.fit(train_images, train_labels,  epochs = 10,
              validation_data = (test_images,test_labels),
              callbacks = [cp_callback])  # pass callback to training   
    return model, checkpoint_dir

In [9]:
def evaluateModel(model):
    loss, acc = model.evaluate(test_images, test_labels)
    print("loss: {:5.2f}% accuracy: {:5.2f}%".format(loss, 100*acc))

In [14]:
def saveModel(model, path):
    model.save(path)
    
def loadModel(model, path):
    model.load(path)
    return model

In [15]:
if __name__ == '__main__':
    isLoadDatabasetsNeeded = True
    if isLoadDatabasetsNeeded:
        loadDatasets()
    model_A = create_model()
    model_A.summary()
    model_A, checkpoint_dir = trainModel(model_A)
    checkpoint_latest = tf.train.latest_checkpoint(checkpoint_dir)
    saveModel(model_A, 'saveRestoreModel.h5')
    evaluateModel(model_A)
    
    model_B = create_model()
    model_B.summary()
    print("before load weights")
    evaluateModel(model_B)
    
    model_B.load_weights(checkpoint_latest)
    print("after load weights")    
    evaluateModel(model_B)
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
checkpoint:trainCheckPoint/cp.ckpt,checkpoint_dir:trainCheckPoint
Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.2210 - acc: 0.6562
Epoch 00001: saving model to trainCheckPoint/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1657 - acc: 0.671

1000/1000 [==============================] - 0s 364us/step
loss:  2.37% accuracy: 11.30%
after load weights
1000/1000 [==============================] - 0s 100us/step
loss:  0.39% accuracy: 87.60%
